Notebook to run sd-webui and Fooocus on hslu gpu hub

## Install packages (run each time)

Check GPU + Cuda version

In [ ]:
!nvidia-smi

Upgrade httpx

In [ ]:
!pip install httpx==0.24.1

Install xformers to speed up inference

In [ ]:
!pip install xformers==0.0.22 huggingface-hub

Downgrade torchvision

In [ ]:
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 --index-url https://download.pytorch.org/whl/cu117

Check if GPU is working

In [ ]:
import torch
torch.cuda.is_available()

## Declare var + functions (run each time)

In [ ]:
import os
import git
import shutil
from pathlib import Path

# Home path (only works on Linux)
# https://stackoverflow.com/a/58988310
home_dir = Path(os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID']))

# Paths
sd_dir = home_dir / 'stable-diffusion-webui'
sd_model_dir = sd_dir / 'models'
sd_extensions_dir = sd_dir / 'extensions'
fooocus_dir = home_dir / 'Fooocus'
fooocus_api_dir = home_dir / 'Fooocus-API'

In [ ]:
# Clone a repo
def clone_repo(git_url, target_dir):
    
    # Extract repo name and create path
    repo_name = git_url.split('/')[-1]

    # Create path
    repo_dir = target_dir / repo_name

    # Clone if does not exist
    if not repo_dir.exists():
        git.Repo.clone_from(git_url, repo_dir)

# FOOOCUS

## Install Fooocus (run once)

**⚠️** Warning: only run the next cell if you want to delete the existing copy of Fooocus **⚠️**

In [ ]:
answer = input("-> Are you sure you want to delete 'Fooocus' [y | n]: ")
if answer == 'y':
    if fooocus_dir.exists():
        shutil.rmtree(fooocus_dir)
        print('Folder deleted')
    else:
        print('Folder not found')
else:
    print('Skipping')

Clone repo

In [ ]:
clone_repo('https://github.com/lllyasviel/Fooocus', home_dir)

Start Fooocus once to initialize all files, generate a few image and stop the execution of the cell.

In [ ]:
!python {fooocus_dir}/entry_with_update.py --share

#### Manually modify the file `Fooocus/user_path_config.txt`
Change
```
    "modelfile_path": "/home/jovyan/Fooocus/models/checkpoints",
    "lorafile_path": "/home/jovyan/Fooocus/models/loras",
```
to 
```
    "modelfile_path": "/home/jovyan/stable-diffusion-webui/models/Stable-diffusion/",
    "lorafile_path": "/home/jovyan/stable-diffusion-webui/models/Lora",
```
and 
```
    "controlnet_models_path": "/home/jovyan/Fooocus/models/controlnet",
```
to
```
    "controlnet_models_path": "/home/jovyan/stable-diffusion-webui/ControlNet",
```
Your home path might be different than `/home/jovyan`

## 📍 Run Fooocus

In [ ]:
!python {fooocus_dir}/entry_with_update.py --share

In [ ]:
clone_repo('https://github.com/konieshadow/Fooocus-API', home_dir)

# SD-WEBUI

## Install sd-webui (run once)

**⚠️** Warning: only run the next cell if you want to delete the existing copy of sd-webui **⚠️**

In [ ]:
answer = input("-> Are you sure you want to delete 'stable-diffusion-webui' [y | n]: ")
if answer == 'y':
    if sd_dir.exists():
        shutil.rmtree(sd_dir)
        print('Folder deleted')
    else:
        print('Folder not found')
else:
    print('Skipping')

Clone and cd into repo

In [ ]:
clone_repo('https://github.com/AUTOMATIC1111/stable-diffusion-webui', home_dir)

#### 1. Manually modify the file `stable-diffusion/webui.sh`
Change `use_venv=1` to `use_venv=0`
#### 2. Manually modify the file `stable-diffusion/webui-user.sh`
Dont forget to uncomment the line!! (delete the `#`)  
change `#export COMMANDLINE_ARGS=""` to  `export COMMANDLINE_ARGS="--xformers --share"`

## Download models (run whenever needed)

In [ ]:
import huggingface_hub

# SD models
local_dir = sd_model_dir / 'Stable-diffusion'

# SD 1.5
repo_id='runwayml/stable-diffusion-v1-5'
filename = 'v1-5-pruned-emaonly.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# SDXL
repo_id='stabilityai/stable-diffusion-xl-base-1.0'
# Base model
filename = 'sd_xl_base_1.0.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)
# VAE
filename = 'sd_xl_base_1.0_0.9vae.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# ControlNet (There's a LOT here, hopefully we have enough disk space 🤞)
local_dir = sd_model_dir / 'ControlNet'

# ControlNet 1.1 
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/ControlNet-v1-1',
    allow_patterns='*.pth',
    local_dir=local_dir
)
# ControlNet XL
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/sd_control_collection',
    local_dir=local_dir
)

## Install sd-webui extensions (run whenever needed)

You can find extensions via the UI then copy paste their repo here

In [ ]:
# Use urls without the .git
git_urls = {
    'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    'https://github.com/Mikubill/sd-webui-controlnet'
}

# Iterate through git repos and clone them
for url in git_urls:
    clone_repo(url, sd_extensions_dir)

## 📍 Run sd-webui

In [ ]:
!cd {sd_dir} && ./webui.sh